# AGVCH Checker

## Get Official Data

In [1]:
import pandas as pd
from datetime import datetime

official_raw = pd.read_csv(f"https://www.agvchapp.bfs.admin.ch/api/communes/snapshot?date={datetime.now().strftime('%d-%m-%Y')}")

# set datatypes
official_raw.Parent = official_raw.Parent.astype("Int64")
official_raw.Inscription = official_raw.Inscription.astype("Int64")
official_raw.BfsCode = official_raw.BfsCode.astype("Int64")

print(official_raw.dtypes)
official_raw.head()

HistoricalCode      int64
BfsCode             Int64
ValidFrom          object
ValidTo           float64
Level               int64
Parent              Int64
Name               object
ShortName          object
Inscription         Int64
Radiation         float64
Rec_Type_fr        object
Rec_Type_de        object
dtype: object


,HistoricalCode,BfsCode,ValidFrom,ValidTo,Level,Parent,Name,ShortName,Inscription,Radiation,Rec_Type_fr,Rec_Type_de
0,1,1,12.09.1848,NaN,1,<NA>,Zürich,ZH,<NA>,NaN,NaN,NaN
1,10053,101,12.09.1848,NaN,2,1,Bezirk Affoltern,Affoltern,100,NaN,NaN,NaN
2,10575,13,12.09.1848,NaN,3,10053,Stallikon,Stallikon,<NA>,NaN,NaN,NaN
3,11742,2,12.09.1848,NaN,3,10053,Affoltern am Albis,Affoltern am Albis,<NA>,NaN,NaN,NaN
4,11801,3,12.09.1848,NaN,3,10053,Bonstetten,Bonstetten,<NA>,NaN,NaN,NaN


## Wrangle Official Data

In [2]:
official = official_raw[official_raw["Level"] == 3]
official = official[["BfsCode", "Name"]]
print(official.dtypes)
official.head()

BfsCode     Int64
Name       object
dtype: object


,BfsCode,Name
2,13,Stallikon
3,2,Affoltern am Albis
4,3,Bonstetten
5,4,Hausen am Albis
6,5,Hedingen


## Wikidata Data

### Define Query Function

In [3]:
import requests
import time

def query_wikidata(query: str, max_retries: int = 5) -> pd.DataFrame:
    
    headers = {
    "User-Agent": "agvch-checker (https://github.com/bequrios/agvch-checker; mailto:bhitz@gmx.net)"}

    url = "https://query.wikidata.org/sparql"
    
    for attempt in range(max_retries):
    
        response = requests.get(url, params={"query": query, "format": "json"}, headers=headers)

        if response.status_code == 200:
            data = response.json()
            return pd.json_normalize(data["results"]["bindings"])
        
        elif response.status_code == 429:
            # too many requests — check Retry-After header
            retry_after = int(response.headers.get("Retry-After", "5"))
            print(f"Rate limited. Waiting {retry_after} seconds before retrying...")
            time.sleep(retry_after)
        
        else:
            # some other error, stop
            response.raise_for_status()
    
    raise Exception("Max retries exceeded")

### Get Wikidata Data

In [4]:
query = """
SELECT * WHERE {
  ?muni wdt:P31 wd:Q70208.
  OPTIONAL { ?muni wdt:P1448 ?name . }
  OPTIONAL { ?muni wdt:P771 ?bfs . }
  MINUS{ ?muni wdt:P31 wd:Q685309. }
}
"""
wiki_raw = query_wikidata(query)

print(wiki_raw.dtypes)
wiki_raw.head()

muni.type        object
muni.value       object
name.xml:lang    object
name.type        object
name.value       object
bfs.type         object
bfs.value        object
dtype: object


,muni.type,muni.value,name.xml:lang,name.type,name.value,bfs.type,bfs.value
0,uri,http://www.wikidata.org/entity/Q66604,de-ch,literal,Risch,literal,1707
1,uri,http://www.wikidata.org/entity/Q66611,und,literal,Oberhofen am Thunersee,literal,934
2,uri,http://www.wikidata.org/entity/Q66613,und,literal,Gurzelen,literal,867
3,uri,http://www.wikidata.org/entity/Q66617,de-ch,literal,Frick,literal,4163
4,uri,http://www.wikidata.org/entity/Q66623,de-ch,literal,Hemmiken,literal,2848


### Wrangle Wikidata

In [5]:
wiki = wiki_raw[["muni.value", "name.value", "name.xml:lang", "bfs.value"]].rename(columns={
    "muni.value": "q",
    "name.value": "name",
    "name.xml:lang": "lang",
    "bfs.value": "bfs"})

wiki["q"] = wiki["q"].str.split("/").str[-1]
wiki["bfsInt"] = wiki["bfs"].astype("Int64")
print(wiki.dtypes)
wiki.head()

q         object
name      object
lang      object
bfs       object
bfsInt     Int64
dtype: object


,q,name,lang,bfs,bfsInt
0,Q66604,Risch,de-ch,1707,1707
1,Q66611,Oberhofen am Thunersee,und,934,934
2,Q66613,Gurzelen,und,867,867
3,Q66617,Frick,de-ch,4163,4163
4,Q66623,Hemmiken,de-ch,2848,2848


## Merge Dataframes

In [6]:
merge = pd.merge(official, wiki, left_on="BfsCode", right_on="bfsInt", how="outer", indicator=True)
merge["_merge"] = merge["_merge"].map({
    "left_only": "official_only",
    "right_only": "wiki_only",
    "both": "both"
})

# new merge columns
merge["_nameOk"] = merge["Name"] == merge["name"]
merge["_bfsOk"] = merge["bfs"] == merge["bfsInt"].astype(str)

# make categorical columns so that the summarizing is consistent (even if there are no False values, count them as zero)
merge["_nameOk"] = pd.Categorical(merge["_nameOk"], categories=[True, False])
merge["_bfsOk"] = pd.Categorical(merge["_bfsOk"], categories=[True, False])

print(merge.dtypes)
merge.head()

BfsCode       Int64
Name         object
q            object
name         object
lang         object
bfs          object
bfsInt        Int64
_merge     category
_nameOk    category
_bfsOk     category
dtype: object


,BfsCode,Name,q,name,lang,bfs,bfsInt,_merge,_nameOk,_bfsOk
0,1,Aeugst am Albis,Q64945,Aeugst am Albis,de-ch,1,1,both,True,True
1,2,Affoltern am Albis,Q68290,Affoltern am Albis,de-ch,2,2,both,True,True
2,3,Bonstetten,Q65510,Bonstetten,de-ch,3,3,both,True,True
3,4,Hausen am Albis,Q65231,Hausen am Albis,de-ch,4,4,both,True,True
4,5,Hedingen,Q65129,Hedingen,de-ch,5,5,both,True,True


## Summarise Merge

In [7]:
# Columns to summarize
columns_to_summarize = ["_merge", "_nameOk", "_bfsOk"]

summary = {col: merge[col].value_counts().to_dict() for col in columns_to_summarize}

summary["_bfsIntUnique"] = merge["bfsInt"].is_unique
summary["_BfsCodeUnique"] = merge["BfsCode"].is_unique

summary
summary["_bfsIntNonUniqueValues"] = merge["BfsCode"][merge["BfsCode"].duplicated(keep=False)].unique().tolist()
summary["_bfsIntNonUniqueValues"] = [x if x is not pd.NA else None for x in summary["_bfsIntNonUniqueValues"]]
summary["_BfsCodeNonUniqueValues"] = merge["bfsInt"][merge["bfsInt"].duplicated(keep=False)].unique().tolist()
summary["_BfsCodeNonUniqueValues"] = [x if x is not pd.NA else None for x in summary["_BfsCodeNonUniqueValues"]]
print(summary)

{'_merge': {'both': 2119, 'wiki_only': 22, 'official_only': 1}, '_nameOk': {True: 1991, False: 151}, '_bfsOk': {True: 2138, False: 4}, '_bfsIntUnique': False, '_BfsCodeUnique': False, '_bfsIntNonUniqueValues': [None, 782, 1331, 2196, 2275, 3291, 6266, 6289, 6630, 6634, 6644], '_BfsCodeNonUniqueValues': [782, 1331, None, 2196, 2275, 3291, 6266, 6289, 6630, 6634, 6644]}


## Missing LINDAS Versions

### Fetch LINDAS Data

In [8]:
# Endpoint URL
url = "https://lindas.cz-aws.net/query"

headers = {
    "Accept": "application/sparql-results+json"
}

# Query string to get BFS codes from Wikidata
query = """
PREFIX schema: <http://schema.org/>
PREFIX vl: <https://version.link/>
SELECT ?identifier WHERE {
?version a vl:Version;
        vl:inVersionedIdentitySet <https://ld.admin.ch/fso/register>;
        schema:identifier ?identifier.
}
"""
# Make the request to the Wikidata SPARQL endpoint
response = requests.get(url, params={'query': query}, headers=headers)
lindas_raw = response.json()
lindas_raw = pd.json_normalize(lindas_raw['results']['bindings'])

# Convert BFS codes to int
lindas_raw["identifier.value"] = lindas_raw["identifier.value"].astype("Int64")

print(lindas_raw.dtypes)
lindas_raw.head()

identifier.datatype    object
identifier.type        object
identifier.value        Int64
dtype: object


,identifier.datatype,identifier.type,identifier.value
0,http://www.w3.org/2001/XMLSchema#integer,literal,13688
1,http://www.w3.org/2001/XMLSchema#integer,literal,10242
2,http://www.w3.org/2001/XMLSchema#integer,literal,10323
3,http://www.w3.org/2001/XMLSchema#integer,literal,14411
4,http://www.w3.org/2001/XMLSchema#integer,literal,10459


### Search Missing

In [9]:
official_l3 = official_raw[official_raw["Level"] == 3]

missing = official_l3[~official_l3["HistoricalCode"].isin(lindas_raw["identifier.value"])]

# Ensure missing values are handled correctly
missing = missing.replace({pd.NA: None, float('nan'): None})

missing = missing[["HistoricalCode", "BfsCode", "ValidFrom", "Name"]]
print(missing.dtypes)
missing.head()

HistoricalCode     int64
BfsCode            Int64
ValidFrom         object
Name              object
dtype: object


,HistoricalCode,BfsCode,ValidFrom,Name
743,16671,2262,01.01.2026,Gurmels
828,16670,2056,01.01.2026,Fétigny-Ménières
943,16672,2525,01.01.2026,Kriegstetten
1446,16673,4095,01.01.2026,Brugg
2279,16669,6831,01.01.2026,Moutier


## Aditional Wikidata Statistics

In [10]:
query = """
SELECT (COUNT(?s) as ?count) WHERE {
  ?s ps:P13740 ?o.
}
"""

wiki_lindas = query_wikidata(query)

print(wiki_lindas.dtypes)
wiki_lindas.head()

count.datatype    object
count.type        object
count.value       object
dtype: object


,count.datatype,count.type,count.value
0,http://www.w3.org/2001/XMLSchema#integer,literal,7


In [11]:
query = """
SELECT (COUNT(?s) as ?count) WHERE {
  ?s ?p ?o.
  ?o prov:wasDerivedFrom/pr:P13740 ?lindas.
}
"""

wiki_lindas_reference = query_wikidata(query)

print(wiki_lindas_reference.dtypes)
wiki_lindas_reference.head()

count.datatype    object
count.type        object
count.value       object
dtype: object


,count.datatype,count.type,count.value
0,http://www.w3.org/2001/XMLSchema#integer,literal,14


## Write Files

In [12]:
import json
from datetime import date

# String for today's date
today_str = date.today().isoformat()  # e.g., '2025-08-04'

# merge CSV
merge.to_csv(f"current_munies/{today_str}.csv", index=False)

# Summary JSON
output = {"current_munies": summary,
          "number_of_lindas_ids_in_wikidata": int(wiki_lindas["count.value"][0]),
          "number_of_references_to_lindas_ids_in_wikidata": int(wiki_lindas_reference["count.value"][0]),
          "missing_lindas_versions": missing.to_dict(orient="records")}

with open(f"summaries/{today_str}.json", "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4)